#Assignment 5


1. Each group must create their branch named group_#_ass_5 (group_1_ass_5) and save their results in the [Assignment_5](https://github.com/alexanderquispe/Diplomado_PUCP/tree/main/Lecture_8/Assignment_5) folder.
2. Generate your API key in Google Cloud. [Follow this link](https://www.youtube.com/watch?v=wxqpvpNKD7A).
3. Allow your API key to use Geocoding API. [Follow this link](https://www.youtube.com/watch?v=d1QGLwie9YU).
4. You have to geolocate observations using addresses of any database of your election and name your **JN** like your branch. Your database should have no more than 100 observations. I suggest sampling your data.
5. The output of your project should look like this table:

|Address|Latitude|Longitude|
|---|---|---|
|Av. Alfonso Ugarte 1227, Cercado de Lima 15001, Peru| -12.055131331273099| -77.04105408467566|

5. Exports your results in a CSV file and save it in the same folder and name it like your branch.
3. Your Pull request should be linked to this issue.
4. All the questions about the assignment should be posted in this issue.

In [3]:
#Packages
import pandas as pd
import os
import urllib.request, json, csv
import numpy as np
from tqdm import tqdm_notebook as tqdm
import requests
import os
import json
import pandas as pd
import csv
import datetime
import dateutil.parser
import unicodedata
import time
import requests

## Parte 1
#!pip install googlemaps
import googlemaps
import pprint
import re

data = pd.read_csv(r"./PCR_data.csv")
data

ModuleNotFoundError: No module named 'googlemaps'

In [ ]:
#Importar base de datos con direcciones de pruebas PCR gratuitas MINSA

In [ ]:
#Generar latitud y longitud - Google API Key

In [ ]:
#Guardar nueva base de datos con campos de latitud y longitud